# 03 딥러닝 - Conv2D
> 1. Convolution 에 대한 이해와 Convolution Layer의 사용법을 익힌다.
> 2. Convolution 필터의 역할을 중심으로 Convolution Layer에 대한 이해를 한다.
> 3. Convolution 연산이 어떻게 이루어지는지 확인한다

# 1) 컨볼루션의 이해

<img src = "./image/tensorflow102_03_01.jpg" width="400px">

### "O" 모양이 존재하는 개수
* 9 : 전체 - 1개 / 위 - 1개 / 아래 - 0개
* 8 : 전체 - 2개 / 위 - 1개 / 아래 - 1개
* 6 : 전체 - 1개 / 위 - 0개 / 아래 - 1개
* 0 : 전체 - 1개 / 위 - 0개 / 아래 - 0개

## Convolution이란?
* 합성곱
* **특정한 패턴의 특징이 어디서 나타나는지 확인하는 도구**

## 특징맵(feature map)
> 특징에 대한 위치정보가 표현된 맵

<img src = "./image/tensorflow102_03_02.jpg" width="200px">

### 가로 filter
<img src = "./image/tensorflow102_03_03.jpg" width="400px">

### 세로 filter

<img src = "./image/tensorflow102_03_04.jpg" width="400px">


In [10]:
pip install tensorflow

     |████████████████████████████████| 198.9 MB 42.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.9 MB 46.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 38.0 MB/s eta 0:00:01
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
     |████████████████████████████████| 15.6 MB 11.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 49.8 MB/s eta 0:00:01
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
     |████████████████████████████████| 462 kB 38.0 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 23.8 MB/s eta 0:00:01
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-macosx_10_9_x86_64.whl (3.5 MB)
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30702 sha256=d4a92658f4be9cdc728d317809f9ebc2261b5d29adf043630c0c2d5c4e7cb305
  Stored in directory: /Users/yubin/Library/Caches/pip/wheels/98/91/04/971b4c587cf47ae952b108949b46926f

Note: you may need to restart the kernel to use updated packages.


In [13]:
# 라이브러리 준비
import tensorflow as tf
import pandas as pd

In [14]:
# 데이터 준비
(independent, dependent), _ = tf.keras.datasets.mnist.load_data()
independent = independent.reshape(60000, 28, 28, 1)
dependent = pd.get_dummies(dependent)
print(independent.shape, dependent.shape)

(60000, 28, 28, 1) (60000, 10)


## conversion layer 2개 추가
1. 필터 셋을 몇 개 사용할 것인가? ex. `3`, `6` : 3개의 필터 셋 사용, 6개의 필터 셋 사용
 * `n`개의 특징 맵을 만듬 = `n`채널의 특징맵
2. 필터셋의 사이즈를 얼마로 할 것인가? ex. `kernal_size=5` : 5 X 5 사이즈의 필터 셋 사용

#### 채널이란?
> 색채 layer

* 3채널 : 컬러 이미지
* 1채널 : 흑백 이미지
* 이미지 shape은 다음과 같이 표기 (이미지 사이즈, 채널)
  ex. (32, 32, 3) : 컬러 사진 / (28, 28, 1) : 흑백 사진

In [15]:
# 모델 생성
X = tf.keras.layers.Input(shape=[28, 28, 1])
H = tf.keras.layers.Conv2D(3, kernel_size=5, activation='swish')(X) # 3개의 특징맵 = 3채널의 특징맵
H = tf.keras.layers.Conv2D(6, kernel_size=5, activation='swish')(H) # 6개의 특징맵 = 6채널의 특징맵
H = tf.keras.layers.Flatten()(H)  # 픽셀 단위로 합친다. 28 X 28 X 1 (3차원 : 3색) 
H = tf.keras.layers.Dense(84, activation='swish')(H)
Y = tf.keras.layers.Dense(10, activation='softmax')(H)
model = tf.keras.models.Model(X, Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')

# 2) 필터의 이해
1. 필터셋은 **3차원 형태로 된 가중치**의 모음
2. 필터셋 하나는 앞선 레이어의 결과인 **"특징맵" 전체를 본다.**
3. **필터셋 개수 만큼** 특징맵을 만든다


**EX 1.** `Conv2D(3, kernel_size=5, activation='swish')`

`F1`  `F2` `F3` ➡️ (`5`, `5`, `a`)  
                                              
전체 필터 셋 : (`3`, `5`, `5`, `a`)




**EX2.** `Conv2D(6, kernel_size=5, activation='swish')`

`F1` `F2` `F3` `F4` `F5` `F6`

전체 필터 셋 : (`6`, `5`, `5`, `a`)

 *`a` : 특징맵의 채널 수 ex. 흑백이면 (`5`, `5`, `1`) / 컬러이면 (`5`, `5`, `3`)*
 


<img src = "./image/tensorflow102_03_05.png" width="600px">


`M1`, `M2`, `M3`

사이즈 : 이미지 크기 - (필터 크기 -1)

* 특징 자동 추출기

`H = tf.keras.layers.Conv2D(6, kernel_size=5, activation='swish')(H)`

이 이미지들이 0 ~ 9까지 중 어느 숫자인지 판단하기 위해 가장 좋은 특징맵 6개를 찾아줘

In [16]:
# 모델 학습
model.fit(independent, dependent, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 21s 11ms/step - loss: 0.5117 - accuracy: 0.9327
Epoch 2/10
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0778 - accuracy: 0.9779
Epoch 3/10
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0507 - accuracy: 0.9860
Epoch 4/10
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0372 - accuracy: 0.9895
Epoch 5/10
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0271 - accuracy: 0.9921
Epoch 6/10
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0239 - accuracy: 0.9934
Epoch 7/10
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0194 - accuracy: 0.9944
Epoch 8/10
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0174 - accuracy: 0.9951
Epoch 9/10
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0159 - accuracy: 0.9958
Epoch 10/10
1875/1875 [==============================] - 19s 10m

In [17]:
# 모델 이용
pred = model.predict(independent[0:5])
pd.DataFrame(pred).round(2)

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [18]:
# 정답 확인
dependent[0:5]

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1


In [19]:
# 모델 확인
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 24, 24, 3)         78        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 6)         456       
_________________________________________________________________
flatten (Flatten)            (None, 2400)              0         
_________________________________________________________________
dense (Dense)                (None, 84)                201684    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                850       
Total params: 203,068
Trainable params: 203,068
Non-trainable params: 0
_______________________________________________________